## Simple AutoEncoder

The principal of autoencoder

![Simple Autoencoder](../_imgs/autoencoder.png)

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import save_image

In [2]:
if not os.path.exists('./save/mlp_ae'):
    os.mkdir('./save/mlp_ae')

In [3]:
def to_img(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x

In [4]:
num_epoches = 100
batch_size = 128
learning_rate = 1e-3
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # whether GPU is supportted

In [5]:
img_transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [6]:
dataset = datasets.MNIST('../_data/mnist', transform=img_transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [7]:
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(28 * 28, 128), nn.ReLU(True),
            nn.Linear(128, 64), nn.ReLU(True),
            nn.Linear(64, 12), nn.ReLU(True),
            nn.Linear(12, 3))
        self.decoder = nn.Sequential(
            nn.Linear(3, 12), nn.ReLU(True),
            nn.Linear(12, 64), nn.ReLU(True),
            nn.Linear(64, 128), nn.ReLU(True),
            nn.Linear(128, 28 * 28),
            nn.Tanh())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [8]:
model = autoencoder().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)

In [9]:
for epoch in range(num_epoches):
    for data in dataloader:
        img, _ = data
        img = img.reshape(img.size(0), -1)
        img = img.to(device)
        # ===================forward=====================
        output = model(img)
        loss = criterion(output, img)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print(f'epoch [{epoch+1}/{num_epoches}], loss:{loss:.4f}')
    if epoch % 10 == 0:
        pic = to_img(output)
        save_image(pic, 'save/mlp_ae/image_{}.png'.format(epoch))

epoch [1/100], loss:0.1979
epoch [2/100], loss:0.1672
epoch [3/100], loss:0.1794
epoch [4/100], loss:0.1626
epoch [5/100], loss:0.1512
epoch [6/100], loss:0.1619
epoch [7/100], loss:0.1495
epoch [8/100], loss:0.1421
epoch [9/100], loss:0.1441
epoch [10/100], loss:0.1465
epoch [11/100], loss:0.1517
epoch [12/100], loss:0.1494
epoch [13/100], loss:0.1287
epoch [14/100], loss:0.1381
epoch [15/100], loss:0.1421
epoch [16/100], loss:0.1505
epoch [17/100], loss:0.1334
epoch [18/100], loss:0.1458
epoch [19/100], loss:0.1268
epoch [20/100], loss:0.1359
epoch [21/100], loss:0.1249
epoch [22/100], loss:0.1300
epoch [23/100], loss:0.1404
epoch [24/100], loss:0.1354
epoch [25/100], loss:0.1327
epoch [26/100], loss:0.1262
epoch [27/100], loss:0.1486
epoch [28/100], loss:0.1331
epoch [29/100], loss:0.1286
epoch [30/100], loss:0.1281
epoch [31/100], loss:0.1220
epoch [32/100], loss:0.1274
epoch [33/100], loss:0.1257
epoch [34/100], loss:0.1303
epoch [35/100], loss:0.1379
epoch [36/100], loss:0.1254
e

In [11]:
torch.save(model.state_dict(), 'save/mlp_ae/sim_autoencoder.pytorch')